In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
slim=tf.contrib.slim
if __name__ == "__main__":
    sys.path.append("../../../")

In [2]:
from notebooks_src.box_encode_decode.ssd.make_anchors_orig import make_anchors_for_one_fmap
from notebooks_src.box_encode_decode.ssd.encode import encode, encode_one_fmap
from notebooks_src.models.configs import configs
from notebooks_src.box_encode_decode.configs import configs as box_configs
from notebooks_src.losses.util import unpack_net_output
configs.update(box_configs)

importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/ssd/make_anchors_orig.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/ssd/encode.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/get_generator.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/generator/generator.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/util.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/generator/batch_fetcher.ipynb
importing Jupyter notebook from ../../../notebooks_src/models/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/config_util.ipynb
importing Jupyter notebook from ../../../notebooks_src/losses/util.ipynb


In [18]:
def compute_loss(y_true, y_pred):
    #TODO: make work for batch size greater than 1
    '''y_true: the boxes
       y_pred: a tensor of Nxfy x fx x k where k = 4*number of anchors + number_of_anchors*num_classes'''
    feat_shape = tuple([d.value for d in list(y_pred.get_shape()[1:3])])
    #anchors = make_anchors_for_one_fmap(img_shape=configs["tensor_input_shape"], feat_shape=feat_shape)
    y_true = tf.squeeze(y_true,axis=0)
    y_pred = tf.squeeze(y_pred,axis=0)
    gclasses, glocalizations, gscores= encode_one_fmap(y_true, feat_shape=feat_shape)
    
    #TODO: reshape to have 96,144,4,4 for localizations and logits
    localizations, logits = unpack_net_output(y_pred, feat_shape)
    return logits, localizations,gclasses, glocalizations, gscores
    return losses(logits, 
                  localizations,
                  gclasses, 
                  glocalizations, 
                  gscores)
    


In [4]:
from notebooks_src.load_data.get_generator import get_generator

In [5]:
gen=get_generator("tr", batch_size=2)

importing Jupyter notebook from ../../../notebooks_src/load_data/datasets/climate/get_data.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/datasets/climate/util.ipynb


In [6]:
im, box = gen.next()

In [4]:
bboxes = tf.placeholder(tf.float32,shape=(1,15,5),name="bboxes")

In [5]:
bboxes

<tf.Tensor 'bboxes:0' shape=(1, 15, 5) dtype=float32>

In [6]:
y_pred = tf.ones((1,96,144,32))

In [19]:
compute_loss(bboxes,y_pred)

(<tf.Tensor 'split_11:1' shape=(96, 144, 16) dtype=float32>,
 <tf.Tensor 'split_11:0' shape=(96, 144, 16) dtype=float32>,
 <tf.Tensor 'while_5/Exit_1:0' shape=(96, 144, 4) dtype=int64>,
 <tf.Tensor 'stack_5:0' shape=(96, 144, 4, 4) dtype=float32>,
 <tf.Tensor 'while_5/Exit_2:0' shape=(96, 144, 4) dtype=float32>)

In [10]:
loss_weights = None

In [16]:
def losses(logits, localisations,
           gclasses, glocalisations, gscores,
           match_threshold=0.5,
           negative_ratio=3.,
           alpha=1.,
           label_smoothing=0.,
           scope='ssd_losses'):
    """Define the SSD network losses.
    """
    return ssd_single_loss(logits, localisations,
                      gclasses, glocalisations, gscores,
                      match_threshold=match_threshold,
                      negative_ratio=negative_ratio,
                      alpha=alpha,
                      label_smoothing=label_smoothing,
                      scope=scope)


def ssd_single_loss(logits, localisations,
           gclasses, glocalisations, gscores,
           match_threshold=0.5,
           negative_ratio=3.,
           alpha=1.,
           label_smoothing=0.,
           scope=None):
    """Loss functions for training the SSD 300 VGG network.

    This function defines the different loss components of the SSD, and
    adds them to the TF loss collection.

    Arguments:
      logits: predictions logits Tensor;
      localisations: localisations Tensor;
      gclasses: groundtruth labels Tensor;
      glocalisations: groundtruth localisation Tensor;
      gscores: groundtruth score Tensor;
    """
    dtype = logits.dtype
    # Determine weights Tensor.
    pmask = gscores > match_threshold
    fpmask = tf.cast(pmask, dtype)
    n_positives = tf.reduce_sum(fpmask)

    # Select some random negative entries.
    # n_entries = np.prod(gclasses[i].get_shape().as_list())
    # r_positive = n_positives / n_entries
    # r_negative = negative_ratio * n_positives / (n_entries - n_positives)

    # Negative mask.
    no_classes = tf.cast(pmask, tf.int32)
    predictions = slim.softmax(logits)
    nmask = tf.logical_and(tf.logical_not(pmask),
                           gscores > -0.5)
    fnmask = tf.cast(nmask, dtype)
    nvalues = tf.where(nmask,
                       predictions[:, :, :, :, 0],
                       1. - fnmask)
    nvalues_flat = tf.reshape(nvalues, [-1])
    # Number of negative entries to select.
    n_neg = tf.cast(negative_ratio * n_positives, tf.int32)
    n_neg = tf.maximum(n_neg, tf.size(nvalues_flat) // 8)
    n_neg = tf.maximum(n_neg, tf.shape(nvalues)[0] * 4)
    max_neg_entries = 1 + tf.cast(tf.reduce_sum(fnmask), tf.int32)
    n_neg = tf.minimum(n_neg, max_neg_entries)

    val, idxes = tf.nn.top_k(-nvalues_flat, k=n_neg)
    minval = val[-1]
    # Final negative mask.
    nmask = tf.logical_and(nmask, -nvalues > minval)
    fnmask = tf.cast(nmask, dtype)

    # Add cross-entropy loss.

    #cross entropy for positives
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                          labels=gclasses)
    loss = tf.losses.compute_weighted_loss(loss, fpmask)
    l_cross_pos = loss

    #cross entropy for negatives
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                          labels=no_classes)
    loss = tf.losses.compute_weighted_loss(loss, fnmask)
    l_cross_neg = loss

    

    # Weights Tensor: positive mask + random negative.
    weights = tf.expand_dims(alpha * fpmask, axis=-1)
    loss = custom_layers.abs_smooth(localisations - glocalisations)
    loss = tf.losses.compute_weighted_loss(loss, weights)
    l_loc = loss
    
    final_loss = l_cross_pos + l_cross_neg  + l_loc
    return final_loss

In [12]:
# =========================================================================== #
# SSD loss function.
# =========================================================================== #
def ssd_losses(logits, localisations,
           gclasses, glocalisations, gscores,
           match_threshold=0.5,
           negative_ratio=3.,
           alpha=1.,
           label_smoothing=0.,
           scope=None):
    """Loss functions for training the SSD 300 VGG network.

    This function defines the different loss components of the SSD, and
    adds them to the TF loss collection.

    Arguments:
      logits: (list of) predictions logits Tensors;
      localisations: (list of) localisations Tensors;
      gclasses: (list of) groundtruth labels Tensors;
      glocalisations: (list of) groundtruth localisations Tensors;
      gscores: (list of) groundtruth score Tensors;
    """
    with tf.name_scope(scope, 'ssd_losses'):
        l_cross_pos = []
        l_cross_neg = []
        l_loc = []
        for i in range(len(logits)):
            dtype = logits[i].dtype
            with tf.name_scope('block_%i' % i):
                # Determine weights Tensor.
                pmask = gscores[i] > match_threshold
                fpmask = tf.cast(pmask, dtype)
                n_positives = tf.reduce_sum(fpmask)

                # Select some random negative entries.
                # n_entries = np.prod(gclasses[i].get_shape().as_list())
                # r_positive = n_positives / n_entries
                # r_negative = negative_ratio * n_positives / (n_entries - n_positives)

                # Negative mask.
                no_classes = tf.cast(pmask, tf.int32)
                predictions = slim.softmax(logits[i])
                nmask = tf.logical_and(tf.logical_not(pmask),
                                       gscores[i] > -0.5)
                fnmask = tf.cast(nmask, dtype)
                nvalues = tf.where(nmask,
                                   predictions[:, :, :, :, 0],
                                   1. - fnmask)
                nvalues_flat = tf.reshape(nvalues, [-1])
                # Number of negative entries to select.
                n_neg = tf.cast(negative_ratio * n_positives, tf.int32)
                n_neg = tf.maximum(n_neg, tf.size(nvalues_flat) // 8)
                n_neg = tf.maximum(n_neg, tf.shape(nvalues)[0] * 4)
                max_neg_entries = 1 + tf.cast(tf.reduce_sum(fnmask), tf.int32)
                n_neg = tf.minimum(n_neg, max_neg_entries)

                val, idxes = tf.nn.top_k(-nvalues_flat, k=n_neg)
                minval = val[-1]
                # Final negative mask.
                nmask = tf.logical_and(nmask, -nvalues > minval)
                fnmask = tf.cast(nmask, dtype)

                # Add cross-entropy loss.
                with tf.name_scope('cross_entropy_pos'):
                    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits[i],
                                                                          labels=gclasses[i])
                    loss = tf.losses.compute_weighted_loss(loss, fpmask)
                    l_cross_pos.append(loss)

                with tf.name_scope('cross_entropy_neg'):
                    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits[i],
                                                                          labels=no_classes)
                    loss = tf.losses.compute_weighted_loss(loss, fnmask)
                    l_cross_neg.append(loss)

                # Add localization loss: smooth L1, L2, ...
                with tf.name_scope('localization'):
                    # Weights Tensor: positive mask + random negative.
                    weights = tf.expand_dims(alpha * fpmask, axis=-1)
                    loss = custom_layers.abs_smooth(localisations[i] - glocalisations[i])
                    loss = tf.losses.compute_weighted_loss(loss, weights)
                    l_loc.append(loss)

        # Additional total losses...
        with tf.name_scope('total'):
            total_cross_pos = tf.add_n(l_cross_pos, 'cross_entropy_pos')
            total_cross_neg = tf.add_n(l_cross_neg, 'cross_entropy_neg')
            total_cross = tf.add(total_cross_pos, total_cross_neg, 'cross_entropy')
            total_loc = tf.add_n(l_loc, 'localization')

            # Add to EXTRA LOSSES TF.collection
            tf.add_to_collection('EXTRA_LOSSES', total_cross_pos)
            tf.add_to_collection('EXTRA_LOSSES', total_cross_neg)
            tf.add_to_collection('EXTRA_LOSSES', total_cross)
            tf.add_to_collection('EXTRA_LOSSES', total_loc)